In [1]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [4]:
df['point_bins'] = pd.cut(
    df['total_points_per_game'],
    [0, 30, 35, 40, 45, float("inf")],
    right=False,
    labels=['< 30', '30 - 35', '35 - 40', '40 - 45', "> 45"]
)

In [5]:
players_list = list(np.unique(df[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].values))

In [6]:
def get_player_stats(player, df):
    player_matches = df[
        np.where(
            np.logical_or.reduce([df[i] == player for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]),
            True,
            False
        )
    ].copy()
    
    player_matches["belongs_to"] = np.where(
        np.logical_or(
            *[player_matches[i] == player for i in ["team_1_player_1", "team_1_player_2"]]
        ),
        'team_1',
        'team_2'
    )
    
    player_matches['player_team_points'] = np.where(
        player_matches["belongs_to"] == 'team_1',
        player_matches["points_team_1"],
        player_matches["points_team_2"]
    )
    
    player_matches['result'] = np.where(player_matches.belongs_to == player_matches.winner, "win", "loss")
    player_matches['is_win'] = np.where(player_matches.result == "win", 1, 0)
    
    return player_matches

In [7]:
team_1 = ['raghotham', 'nithin']
team_2 = ['ajay', 'vinay']

In [12]:
player_matches = get_player_stats(team_1[0], df)
player_matches['partner'] = np.where(
    player_matches["belongs_to"] == 'team_1',
    np.where(
        player_matches["team_1_player_1"] == team_1[0],
        player_matches["team_1_player_2"],
        player_matches["team_1_player_1"]
    ),
    np.where(
        player_matches["team_2_player_1"] == team_1[0],
        player_matches["team_2_player_2"],
        player_matches["team_2_player_1"]
    ),
)

In [13]:
player_matches

,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins,belongs_to,player_team_points,result,is_win,partner
0,12/27/2022,vinay,raghotham,ajay,aakarsh,15,21,isro match point,team_2,6,36,35 - 40,team_1,15,loss,0,vinay
1,12/28/2022,vinay,raghotham,ajay,sameer,25,27,isro match point,team_2,2,52,> 45,team_1,25,loss,0,vinay
2,12/28/2022,vinay,raghotham,ajay,srinidhi,21,18,isro match point,team_1,3,39,35 - 40,team_1,21,win,1,vinay
3,12/28/2022,sameer,raghotham,vinay,srinidhi,14,21,isro match point,team_2,7,35,35 - 40,team_1,14,loss,0,sameer
4,12/28/2022,sameer,raghotham,ajay,srinidhi,21,18,isro match point,team_1,3,39,35 - 40,team_1,21,win,1,sameer
5,12/28/2022,sameer,raghotham,ajay,vinay,24,26,isro match point,team_2,2,50,> 45,team_1,24,loss,0,sameer
6,12/28/2022,srinidhi,raghotham,ajay,vinay,19,21,isro match point,team_2,2,40,40 - 45,team_1,19,loss,0,srinidhi
7,12/29/2022,srinidhi,raghotham,ajay,prateek,21,16,isro match point,team_1,5,37,35 - 40,team_1,21,win,1,srinidhi
8,12/29/2022,srinidhi,raghotham,ajay,prateek,22,24,isro match point,team_2,2,46,> 45,team_1,22,loss,0,srinidhi
9,12/29/2022,srinidhi,raghotham,ajay,prateek,22,20,isro match point,team_1,2,42,40 - 45,team_1,22,win,1,srinidhi


In [60]:
head_2_head_df = player_matches[np.where(
    player_matches['belongs_to'] == 'team_1',
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_2_player_1'].isin(team_2),
        player_matches['team_2_player_2'].isin(team_2),
    ]),
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_1_player_1'].isin(team_2),
        player_matches['team_1_player_2'].isin(team_2),
    ])
)].copy().reset_index()

head_2_head_df["other_team_points"] = head_2_head_df["total_points_per_game"] - head_2_head_df["player_team_points"]

In [89]:
head_2_head_df

,index,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins,belongs_to,player_team_points,result,is_win,partner,other_team_points
0,16,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin,21
1,18,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin,21
2,20,12/30/2022,nithin,raghotham,ajay,vinay,20,22,prakash badminton academy,team_2,2,42,40 - 45,team_1,20,loss,0,nithin,22
3,22,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin,21
4,36,12/26/2022,nithin,raghotham,ajay,vinay,21,15,match point - gublaala,team_1,6,36,35 - 40,team_1,21,win,1,nithin,15
5,37,12/26/2022,nithin,raghotham,ajay,vinay,18,21,match point - gublaala,team_2,3,39,35 - 40,team_1,18,loss,0,nithin,21
6,38,12/26/2022,nithin,raghotham,ajay,vinay,16,21,match point - gublaala,team_2,5,37,35 - 40,team_1,16,loss,0,nithin,21
7,41,12/26/2022,nithin,raghotham,ajay,vinay,21,13,match point - gublaala,team_1,8,34,30 - 35,team_1,21,win,1,nithin,13


In [62]:
longest_game = head_2_head_df.iloc[head_2_head_df["total_points_per_game"].idxmax(), :].to_dict()
longest_game

{'index': 20,
 'date': '12/30/2022',
 'team_1_player_1': 'nithin',
 'team_1_player_2': 'raghotham',
 'team_2_player_1': 'ajay',
 'team_2_player_2': 'vinay',
 'points_team_1': 20,
 'points_team_2': 22,
 'venue': 'prakash badminton academy',
 'winner': 'team_2',
 'margin': 2,
 'total_points_per_game': 42,
 'point_bins': '40 - 45',
 'belongs_to': 'team_1',
 'player_team_points': 20,
 'result': 'loss',
 'is_win': 0,
 'partner': 'nithin',
 'other_team_points': 22}

In [116]:
head_2_head_stats = {
    "total_games": head_2_head_df.shape[0],
    "team_1_wins": head_2_head_df["is_win"].sum(),
    "average_ppg": head_2_head_df["total_points_per_game"].mean(),
    "average_margin_of_victory": head_2_head_df["margin"].mean(),
    "avg_team1_pts": head_2_head_df["player_team_points"].mean(),
    "avg_team2_pts": head_2_head_df["other_team_points"].mean(),
#     "avg_ppg_in_loss"
    "longest_game": f"{longest_game['total_points_per_game']} points: ({longest_game['team_1_player_1']}, {longest_game['team_1_player_2']}) {longest_game['points_team_1']} - {longest_game['points_team_2']} ({longest_game['team_2_player_1']}, {longest_game['team_2_player_2']}) on {longest_game['date']} at {longest_game['venue']}"
}

head_2_head_stats["team_2_wins"] = head_2_head_stats["total_games"] - head_2_head_stats["team_1_wins"]

In [117]:
head_2_head_stats

{'total_games': 8,
 'team_1_wins': 2,
 'average_ppg': 36.625,
 'average_margin_of_victory': 5.625,
 'avg_team1_pts': 17.25,
 'avg_team2_pts': 19.375,
 'longest_game': '42 points: (nithin, raghotham) 20 - 22 (ajay, vinay) on 12/30/2022 at prakash badminton academy',
 'team_2_wins': 6}

In [72]:
df2 = pd.DataFrame([head_2_head_stats]).T
df2

,0
total_games,8
team_1_wins,2
average_ppg,36.625
average_margin_of_victory,5.625
avg_team1_pts,17.25
avg_team2_pts,19.375
longest_game,"42 points: (nithin, raghotham) 20 - 22 (ajay, ..."
team_2_wins,6


In [88]:
table = go.Table(
    columnwidth=[2, 3],
    header=dict(values=["Metric", "Detail"], align="left", height=40, font=dict(size=20)),
    cells=dict(
        values=[list(head_2_head_stats.keys()), list(head_2_head_stats.values())],
        align="left",
        height=40,
        font=dict(size=15)
    )
)
fig = go.Figure(table)
fig

In [98]:
go.Figure(
    go.Pie(
        labels=[' & '.join(team_1), ' & '.join(team_2)],
        values=[5, 8],
        hole=0.3,
        marker=dict(colors=["#b5de2b", "lightslategrey"])
    ),
)

In [113]:
fig = go.Figure(
    go.Indicator(
        mode="gauge+number+delta",
        value=head_2_head_stats['average_ppg'],
        title="Average Points per Game",
        delta={"reference": 38.3}
    )
)

fig.update_traces(gauge_bar_color="#66bb6a")